In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [ ]:
data=pd.read_csv("../input/tabular-playground-series-jun-2022/data.csv",index_col='row_id')
data.head()

In [ ]:
sample=pd.read_csv("../input/tabular-playground-series-jun-2022/sample_submission.csv")
sample.head()

In [ ]:
data.isnull().sum().sum()

In [ ]:
data.columns.isnull().sum()

In [ ]:
data_train=data.copy()

In [ ]:
submission=pd.DataFrame(columns=['row_col','value'])
import lightgbm as lgb

for i in range(len(data_train.columns)):
    data_pred=data_train.copy()
    data_pred_index=data_pred[data.iloc[:,i].isnull()].index
    data_pred=np.delete(data_pred[data.iloc[:,i].isnull()].values,i,axis=1)
    if len(data_pred)>0:
        X_train=np.delete(data_train[data_train.iloc[:,i].notnull()].values,i,axis=1)
        y_train=data_train[data_train.iloc[:,i].notnull()].iloc[:,i].values
    
        print(f'-{data_train.columns[i]}-----')
        ##
              
        lgb_train=lgb.Dataset(X_train,y_train)
              
        #
        params = {'task':'train','boosting_type':'gbdt','objective':'regression','metric':'root_mean_squared_error','learning_rate':0.1,'num_leaves':220,'min_data_in_leaf':100,}
              
        callbacks=[
                            lgb.early_stopping(100,verbose=True),
              ]
        #
        evaluation_results = {}
        model_lgb = lgb.train(params,lgb_train,num_boost_round=200,valid_sets=[lgb_train],evals_result=evaluation_results,callbacks=callbacks,verbose_eval=-1)
        pred_pred = model_lgb.predict(data_pred)
        df_pred_pred=pd.concat([pd.Series(data_pred_index,name='row-col').astype('str')+'-'+data_train.columns[i],pd.Series(pred_pred,name='value')],axis=1)
        submission = pd.concat([submission,df_pred_pred])
              
              
          

In [ ]:
submission

In [ ]:
submission.to_csv('../working/submission-lightGBM.csv',index=False)